In [2]:
# import all of the necessary libraries 
import pandas as pd
import numpy as np
import seaborn as sns
from __future__ import division
import matplotlib.pylab as plt
%matplotlib inline

In [3]:
# Get TPM using tpm function 
def counts_to_tpm(counts_matrix):
    data_cols = counts_matrix.columns[1:]
    rpk = counts_matrix[data_cols].divide((counts_matrix['Length']/1000), axis=0)
    sums_per_million = rpk.sum(axis=0)/(10**6)
    tpm = rpk.divide(sums_per_million)
    
    return tpm

In [4]:
# read in the positve_iDOGS file 
junc=pd.read_table('positive_iDOGS.txt',comment='#',index_col=0)
junc.head(3)

,Chr,Start,End,Strand,Length,/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam,/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam
Geneid,,,,,,,,,,,
WBGene00022276,I,10403,10423,+,21,0,0,0,0,0,0
WBGene00022275,I,43723,43743,+,21,1,1,0,0,0,1
WBGene00044345,I,47462,47482,+,21,16,17,32,3,3,2


In [5]:
junc=junc[junc['Strand']=="+"]

In [6]:
# rename columns to something more palatable 
junc=junc.rename(columns={'/home/wschrein/scratch/nuevo/att_1/bams/Sample1Aligned.out.sam.sorted.bam':'ctr_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample2Aligned.out.sam.sorted.bam':'ctr_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample3Aligned.out.sam.sorted.bam':'ctr_4',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample4Aligned.out.sam.sorted.bam':'hs_1',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample5Aligned.out.sam.sorted.bam':'hs_3',
                          '/home/wschrein/scratch/nuevo/att_1/bams/Sample6Aligned.out.sam.sorted.bam':'hs_4'
                          })
junc.head(1)

,Chr,Start,End,Strand,Length,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,,,,,,
WBGene00022276,I,10403,10423,+,21,0,0,0,0,0,0


In [7]:
# make an info column of things that you might need later 
info=junc.reset_index()
info=info[['Geneid','Chr','Start','End']]

In [8]:
# now get rid of them 
junc.drop(['Chr','Start','End'],inplace=True,axis=1)

In [14]:
junc_tpm=counts_to_tpm(junc) # get the tpm for all of the reads 
junc_tpm.drop('Length',axis=1,inplace=True) # no longer need the length column

In [15]:
junc_tpm.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00022276,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
data_cols=junc_tpm.columns[0:]
data_cols

Index([u'ctr_1', u'ctr_3', u'ctr_4', u'hs_1', u'hs_3', u'hs_4'], dtype='object')

In [17]:
junc_tpm_filterd = junc_tpm.loc[junc_tpm[data_cols].sum(axis=1) > 0]

In [18]:
junc_tpm_filterd.head(1)

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4
Geneid,,,,,,
WBGene00022275,15.34731,20.620683,0.0,0.0,0.0,13.462212


In [36]:
junc_tpm_filterd['ctrl_total']= junc_tpm_filterd[junc_tpm_filterd.columns[0:3]].sum(axis=1)

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
junc_tpm_filterd['hs_total']= junc_tpm_filterd[junc_tpm_filterd.columns[3:6]].sum(axis=1)

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
junc_tpm_filterd

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
Geneid,,,,,,,,,
WBGene00022275,15.347310,20.620683,0.000000,0.000000,0.000000,13.462212,35.967992,13.462212,0.364159
WBGene00044345,245.556954,350.551603,593.504832,25.063704,55.635919,26.924423,1189.613389,107.624045,0.090394
WBGene00021677,30.694619,164.965460,0.000000,0.000000,0.000000,0.000000,195.660080,0.000000,0.000000
WBGene00000812,153.473096,268.068873,315.299442,83.545679,278.179593,175.008750,736.841411,536.734022,0.727438
WBGene00021683,30.694619,0.000000,0.000000,0.000000,0.000000,94.235481,30.694619,94.235481,2.973233
WBGene00021682,61.389238,61.862048,0.000000,16.709136,0.000000,40.386635,123.251286,57.095770,0.459519
WBGene00021676,30.694619,0.000000,92.735130,16.709136,0.000000,0.000000,123.429749,16.709136,0.134286
WBGene00004418,122.778477,20.620683,74.188104,0.000000,0.000000,94.235481,217.587263,94.235481,0.431111
WBGene00018773,0.000000,0.000000,0.000000,0.000000,0.000000,13.462212,0.000000,13.462212,13.462212


In [39]:
junc_tpm_filterd['ctrl_total']=junc_tpm_filterd['ctrl_total'] +1 # need to add one to each replicate 

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
junc_tpm_filterd['ratio']=junc_tpm_filterd.hs_total / junc_tpm_filterd.ctrl_total

/home/wschrein/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
junc_tpm_filterd.head(1) #  

,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
Geneid,,,,,,,,,
WBGene00022275,15.34731,20.620683,0.0,0.0,0.0,13.462212,36.967992,13.462212,0.364159


In [42]:
# Make it so that only two replicates have to be non-zero

_1_3=junc_tpm_filterd[(junc_tpm_filterd['hs_1']!=0) & (junc_tpm_filterd['hs_3']!=0)]
_1_4=junc_tpm_filterd[(junc_tpm_filterd['hs_1']!=0) & (junc_tpm_filterd['hs_4']!=0)]
_3_4=junc_tpm_filterd[(junc_tpm_filterd['hs_3']!=0) & (junc_tpm_filterd['hs_4']!=0)]


In [43]:
df_concat=pd.concat([_1_3,_1_4,_3_4]) # now add them on top of each other 
df_concat.reset_index(inplace=True)
df_concat.drop_duplicates(inplace=True)

In [44]:
hs_filtered_junc=df_concat
df_concat.head(1)

,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00044345,245.556954,350.551603,593.504832,25.063704,55.635919,26.924423,1190.613389,107.624045,0.090394


# change ratio to greater than 2 and see what happens
# This ratio seems to be the best for our purposes

In [45]:
hs_filtered_junc=hs_filtered_junc[hs_filtered_junc['ratio']>2]

In [46]:
hs_filtered_junc.reset_index(inplace=True)

hs_filtered_junc[hs_filtered_junc['Geneid'].str.contains("WBGene00001534")]

,index,Geneid,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
1171,4854,WBGene00001534,0.0,0.0,0.0,0.0,18.545306,188.470962,1.0,207.016268,207.016268


# only focus on the protein coding genes / don't worry about this for now
pcgs=pd.read_table('gtf_protein_simpified.csv',names=['Geneid'])
gene_names=pd.read_csv('c_elegans.PRJNA13758.current.geneIDs.txt',names=['numba','Geneid','common_name','transcript','status'])
gene_names.drop(['numba','status'],axis=1,inplace=True)

In [191]:
bad_plus_dogs=pd.read_csv('bad_dawgs_plus_for_saving.csv',names=['WB_ID'])
bad_plus_dogs.head()
bad_dogs_list=list(bad_plus_dogs['WB_ID'])

IOError: File bad_dawgs_plus_for_saving.csv does not exist

In [35]:
for i in bad_dogs_list:
    if i in list(hs_filtered_junc['Geneid']):
        print("good")
    else:
        print("bad")

NameError: name 'bad_dogs_list' is not defined

In [193]:
# is this good or is this just picking out all genes that are up in HS indescrimateley?!?

In [194]:
#hs_filtered_junc[hs_filtered_junc['hs_total']>100].head()

In [195]:
# current gene IDs 
gene_names=pd.read_csv('c_elegans.PRJNA13758.current.geneIDs.txt',names=['numba','Geneid','common_name','transcript','status']) 
gene_names.drop(['numba','status'],axis=1,inplace=True)

In [196]:
merge_to_get_gene_names=pd.merge(gene_names,hs_filtered_junc,on='Geneid',how='inner')

In [197]:
merge_to_get_gene_names.sort_values('ratio')
# lot's of non-coding stuff here 
; # remove ';' to see full output 

"# remove ';' to see full output "

# Maybe we should filter the things that are not protein coding genes out


In [198]:
# get the list of protein coding genes 
pcgs=pd.read_table('gtf_protein_simpified.csv',names=['Geneid'])
pcgs.head(1) # perfect; 

,Geneid
0,WBGene00010957


In [199]:
pcgs_merge=pd.merge(merge_to_get_gene_names,pcgs,how='inner',on='Geneid')

In [200]:
print(len(pcgs_merge))
pcgs_merge.head() # this is a little better 
# this is more than when we required all three, maybe this is better or worse 

913


,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00000082,adt-1,C02B4.1,5295,0.000000,0.0,0.0,0.000000,18.545306,26.924423,1.000000,45.469729,45.469729
1,WBGene00000097,aip-1,F58E10.4,2991,0.000000,0.0,0.0,8.354568,0.000000,80.773269,1.000000,89.127837,89.127837
2,WBGene00000118,alh-12,Y69F12A.2,4179,0.000000,0.0,0.0,0.000000,18.545306,26.924423,1.000000,45.469729,45.469729
3,WBGene00000120,aly-1,C01F6.5,862,0.000000,0.0,0.0,8.354568,55.635919,0.000000,1.000000,63.990486,63.990486
4,WBGene00000172,aqp-4,F40F9.9,1139,61.389238,0.0,0.0,8.354568,92.726531,175.008750,62.389238,276.089849,4.425280


In [201]:
pcgs_merge[pcgs_merge.Geneid.str.contains("WBGene00012195")]

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
452,WBGene00012195,bath-34,W02A11.5,3630,0.0,0.0,0.0,0.0,18.545306,13.462212,1.0,32.007518,32.007518


In [202]:
for i in bad_dogs_list:                          # still filters 
    if i in list(pcgs_merge['Geneid']):
        print("good")
    else:
        print("bad")

NameError: name 'bad_dogs_list' is not defined

In [203]:
bad_dogs_list[4] # this is the one that doesnt get filtered out: why? 

# what does it look like; ncRNA upstream this is likley why we didn't detect it

NameError: name 'bad_dogs_list' is not defined

In [204]:
pcgs_merge[pcgs_merge['Geneid'].str.contains("WBGene00001534")]

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
69,WBGene00001534,gcy-7,F52E1.4,4854,0.0,0.0,0.0,0.0,18.545306,188.470962,1.0,207.016268,207.016268


In [205]:
pcgs_merge.head(1)

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio
0,WBGene00000082,adt-1,C02B4.1,5295,0.0,0.0,0.0,0.0,18.545306,26.924423,1.0,45.469729,45.469729


In [206]:
# ok so what are all of the others genes are the false positive or what 

# let's see how many are in the list of up genes 

# primero hay que copiar la lista aqui! 


In [207]:
list_of_up_df=pd.read_csv('unfiltered_list_of_up.txt') # this must be changed to include all the lists of up
list_of_up_df.drop('Unnamed: 0',axis=1,inplace=True)
list_of_up_df.head(1)

,WB_ID
0,WBGene00000002


In [208]:
pwd

u'/home/wschrein/00001_DOGS_Filter/D._Filter_Using_Junction_and_IR'

In [132]:
list_of_up=list(list_of_up_df['WB_ID'])

In [133]:
cuantas_veces=[]
for i in pcgs_merge['Geneid']:
    if i in list_of_up:
        cuantas_veces.append("si")
    else:
        cuantas_veces.append("no")

pcgs_merge['up_in_hs']=cuantas_veces

In [134]:
cuantas_veces.count("si") # entoces cuales son que son up pero no pertenece a mi lista de perrros malos 
                          # pues solo nos importamos los que estàn en la lista de "positive strand"


192

In [135]:
print(cuantas_veces.count('si'))

print(cuantas_veces.count('no')) 

192
721


In [136]:
pcgs_merge[pcgs_merge['up_in_hs']=='si'].head()

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,up_in_hs
0,WBGene00000082,adt-1,C02B4.1,5295,0.00000,0.000000,0.0,0.000000,18.545306,26.924423,1.000000,45.469729,45.469729,si
1,WBGene00000097,aip-1,F58E10.4,2991,0.00000,0.000000,0.0,8.354568,0.000000,80.773269,1.000000,89.127837,89.127837,si
3,WBGene00000120,aly-1,C01F6.5,862,0.00000,0.000000,0.0,8.354568,55.635919,0.000000,1.000000,63.990486,63.990486,si
7,WBGene00000211,asm-1,B0252.2,1901,0.00000,0.000000,0.0,8.354568,0.000000,26.924423,1.000000,35.278991,35.278991,si
10,WBGene00000228,atn-1,W04D2.1,4955,15.34731,20.620683,0.0,0.000000,92.726531,228.857597,36.967992,321.584128,8.698988,si


In [137]:
pcgs_merge.sort_values('ratio',ascending=False).head()

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,up_in_hs
772,WBGene00021599,srt-42,Y46H3A.1,1039,0.0,0.0,0.0,10576.882911,9791.921665,6717.643574,1.0,27086.448150,27086.448150,si
264,WBGene00007723,NaN,C25F9.2,1266,0.0,0.0,0.0,3584.109612,5155.595119,5209.875878,1.0,13949.580609,13949.580609,si
284,WBGene00008144,gasr-8,C47E8.6,1215,0.0,0.0,0.0,2239.024186,3430.881644,2638.593468,1.0,8308.499298,8308.499298,si
57,WBGene00001076,dpy-17,F54D8.1,585,0.0,0.0,0.0,258.991604,1984.347762,1103.901349,1.0,3347.240714,3347.240714,si
112,WBGene00002214,klc-1,M7.2,903,0.0,0.0,0.0,133.673086,1427.988576,1480.843273,1.0,3042.504935,3042.504935,no


In [138]:
# cls-2	 DOG but it is functional! ; are they all like this? could be iDOG but is spliced correctly 
# col-88	 just a little in 5' not iDOG

# Can we filter further with Intron Retention Score

In [139]:
# get IR score from Desktop; Genes with high IR score 

ir=pd.read_csv('s2_scores.csv')
ir.rename(columns={'WB_ID':'Geneid','ratio':'IR_score'},inplace=True)
ir.drop(["Unnamed: 0"],inplace=True,axis=1)

ir_in_hs=[]
for i in ir['Geneid']:
    if i in list_of_up:
        ir_in_hs.append("yes")
    else:
        ir_in_hs.append("no")
        
ir['up_in_hs']=ir_in_hs

ir=ir[ir['up_in_hs']=='yes']



In [140]:
ir.head(1)

,Geneid,IR_score,up_in_hs
0,WBGene00000002,0.064036,yes


In [141]:
pcgs_merge_IR=pd.merge(pcgs_merge,ir,on='Geneid',how='inner')

In [142]:
len(pcgs_merge_IR)

189

In [143]:
pcgs_merge[pcgs_merge['Geneid'].str.contains("WBGene00001534")]

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,up_in_hs
69,WBGene00001534,gcy-7,F52E1.4,4854,0.0,0.0,0.0,0.0,18.545306,188.470962,1.0,207.016268,207.016268,si


In [144]:
pcgs_merge_IR[pcgs_merge_IR.Geneid.str.contains("WBGene00044180")]

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,up_in_hs_x,IR_score,up_in_hs_y
181,WBGene00044180,NaN,C47E8.11,1214,76.736548,0.0,55.641078,4879.06763,9217.017173,8050.40252,133.377626,22146.487323,166.043496,si,0.365918,yes


In [145]:
cutoff_pcgs_merge_IR=pcgs_merge_IR[pcgs_merge_IR['IR_score']> 0.4]
cutoff_pcgs_merge_IR

,Geneid,common_name,transcript,index,ctr_1,ctr_3,ctr_4,hs_1,hs_3,hs_4,ctrl_total,hs_total,ratio,up_in_hs_x,IR_score,up_in_hs_y
19,WBGene00001534,gcy-7,F52E1.4,4854,0.000000,0.000000,0.000000,0.000000,18.545306,188.470962,1.000000,207.016268,207.016268,si,0.683967,yes
55,WBGene00006385,taf-4,R119.6,6,122.778477,185.586143,241.111338,8981.160449,8104.298802,9315.850408,550.475958,26401.309659,47.960877,si,0.523577,yes
71,WBGene00007932,zip-5,C34D1.5,4972,0.000000,0.000000,0.000000,0.000000,55.635919,67.311058,1.000000,122.946976,122.946976,si,0.550992,yes
72,WBGene00008144,gasr-8,C47E8.6,1215,0.000000,0.000000,0.000000,2239.024186,3430.881644,2638.593468,1.000000,8308.499298,8308.499298,si,0.474011,yes
76,WBGene00008395,NaN,D1086.8,1200,0.000000,0.000000,0.000000,258.991604,222.543674,201.933174,1.000000,683.468451,683.468451,si,0.526916,yes
81,WBGene00008497,NaN,F01D5.7,488,0.000000,0.000000,0.000000,200.509629,445.087348,525.026251,1.000000,1170.623228,1170.623228,si,0.568416,yes
86,WBGene00009182,NaN,F26H11.4,493,445.071979,61.862048,0.000000,1161.284933,871.629391,538.488463,507.934027,2571.402786,5.062474,si,0.440760,yes
89,WBGene00009585,cal-7,F40F9.8,1138,15.347310,0.000000,0.000000,1361.794561,945.810615,780.808271,16.347310,3088.413448,188.924876,si,0.754826,yes
92,WBGene00009824,gpdh-1,F47G4.3,239,61.389238,0.000000,18.547026,91.900246,74.181225,0.000000,80.936264,166.081471,2.052003,si,0.443693,yes
105,WBGene00012003,NaN,T24H10.5,3896,0.000000,0.000000,0.000000,0.000000,18.545306,53.848846,1.000000,72.394152,72.394152,si,1.950968,yes


In [146]:
for i in bad_dogs_list:
    if i in list(cutoff_pcgs_merge_IR['Geneid']):
        print("good")
    else:
        print("bad")

NameError: name 'bad_dogs_list' is not defined

In [147]:
# does NOT contain these are the ones that are being filtered out that are not in the bad dog lists 
# we should have two extras then?? 

pattern = '|'.join(bad_dogs_list)
zz=cutoff_pcgs_merge_IR[~cutoff_pcgs_merge_IR.Geneid.str.contains(pattern)]

cutoff_pcgs_merge_IR[~cutoff_pcgs_merge_IR.Geneid.str.contains(pattern)]

# so what is going on with these genes then 
# should they be filtered out as well 
# grsp-1 is a bad dawg
# Y38H6C.8 tambien es malisimo 
# T27E4.1 tambien es un perrito 
# Y37F4.1 lo mismo
# ZC21.10 tambien es malo 


# show this to amy R02F11.1 # this one seems to be the only false positive 

###### is there anything else new being filtered out now that we are being a bit less restrictive since we allowed two/
###### ok let's see if we are filtering shit or good stuff 

# gcy-7 good filter!!!
# ncr-2 bad filter; but probably not really up so maybe it is ok that it is filtered out 
# D1086.8 (hmmm) defiteley iDOG but somewhat spliced
# jun-1 no sé show to amy (not my fault exon is up)
#WBGene00012621 Y38H6C good filter 
#ZK930.5 good filter
#C06A6.2 no sè show to amy
#pcrg-1 E04F6.2 hmm
#otpl-4	 good filter wasnt in list of up before
# T27E4.1 good filter 
# Y37F4.1 good filter

#ZC196.4good filter

# Y38H6C.25 good filter 


# so this seems like a good filter to apply
# did it filter out the genes that I had manually suggested ie only in two reps or not in original list of up genes 



NameError: name 'bad_dogs_list' is not defined

In [148]:
# the new ones 
# nhr-43 could go either way 
# gpdh-1 same most likley real with bad IR
# clec-238 is bad 

# therefore maybe we should change the cutoff to greater than 4 



In [149]:
print(len(list(zz['Geneid']))) # doesnt matter 

NameError: name 'zz' is not defined

In [150]:
#contains
pattern = '|'.join(bad_dogs_list)
cutoff_pcgs_merge_IR[cutoff_pcgs_merge_IR.Geneid.str.contains(pattern)]

NameError: name 'bad_dogs_list' is not defined

In [151]:
print(len(cutoff_pcgs_merge_IR[cutoff_pcgs_merge_IR.Geneid.str.contains(pattern)]))
print(len(bad_dogs_list))

# so we seem to filter out all of the bad dogs

NameError: name 'pattern' is not defined

In [152]:
bad_plus_filter=cutoff_pcgs_merge_IR[['Geneid']]

In [153]:
len(bad_plus_filter) # this results in five less being filtered out 

29

In [154]:
len(bad_plus_filter)

29

In [156]:
bad_plus_filter.to_csv('bad_plus_filter.csv')

19     False
55     False
71     False
72     False
76     False
81     False
86     False
89     False
92     False
105    False
108    False
113    False
114    False
115    False
119    False
122    False
130    False
137    False
138    False
147    False
152    False
153    False
157    False
165    False
168    False
174    False
177    False
183    False
187    False
Name: Geneid, dtype: bool